# Fed Balance Sheet
* https://www.federalreserve.gov/monetarypolicy/bst_recenttrends.htm
* Update every thursday
* Interactive Chart
* Parse xml data form
* Replace table "fed_balance"

In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from sqlalchemy import create_engine
import os

# Make an HTTP request to retrieve the XML data
response = requests.get('https://www.federalreserve.gov/data.xml')

# Parse the XML data
root = ET.fromstring(response.text)

# Find the chart title="Total Assets of the Federal Reserve" since there are multiple charts in that xml
total_asset = root.find('./chart[@title="Total Assets of the Federal Reserve"]')

# Extract the date and value from the specific chart
data=[]
for observation in total_asset.findall('./series/observation'):
    dt = observation.get('index')
    balance = observation.get('value')
    data.append({'dt': dt, 'balance':balance})
    
# Store the data in a dataframe
df = pd.DataFrame(data)
df['dt'] = pd.to_datetime(df['dt'])
df['dt'] = df['dt'].dt.strftime('%Y%m%d')

# Connect to the database
sql_username = os.environ.get('SQL_USERNAME')
sql_password = os.environ.get('SQL_PASSWORD')
sql_host = os.environ.get('SQL_HOST')
sql_database = os.environ.get('SQL_DATABASE')
engine = create_engine(f'mysql://{sql_username}:{sql_password}@{sql_host}/{sql_database}')

# define SQL command to truncate table
table_name = 'fed_balance'
truncate_cmd = f'TRUNCATE TABLE {table_name};'

# execute SQL command to truncate table
with engine.connect() as conn:
    conn.execute(truncate_cmd)

# Upload the data to the "fed_asset" table
df.to_sql('fed_balance', con=engine, if_exists='append', index=False)


environ({'ACSETUPSVCPORT': '23210', 'ACSVCPORT': '17532', 'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\wshaw\\AppData\\Roaming', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPUTERNAME': 'LAPTOP-EF0A6SNV', 'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe', 'CONDA_BAT': 'C:\\Users\\wshaw\\anaconda3\\condabin\\conda.bat', 'CONDA_DEFAULT_ENV': 'base', 'CONDA_EXE': 'C:\\Users\\wshaw\\anaconda3\\Scripts\\conda.exe', 'CONDA_PREFIX': 'C:\\Users\\wshaw\\anaconda3', 'CONDA_PROMPT_MODIFIER': '(base) ', 'CONDA_PYTHON_EXE': 'C:\\Users\\wshaw\\anaconda3\\python.exe', 'CONDA_SHLVL': '1', 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData', 'EFC_10672': '1', 'HOMEDRIVE': 'C:', 'HOMEPATH': '\\Users\\wshaw', 'LOCALAPPDATA': 'C:\\Users\\wshaw\\AppData\\Local', 'LOGONSERVER': '\\\\LAPTOP-EF0A6SNV', 'NUMBER_OF_PROCESSORS': '16', 'ONEDRIVE': 'C:\\Users

818